https://www.machinio.com/

### Objectives
Scrape this data:

<li>Make - Combilift or AisleMaster</li>
<li>Model</li>
<li>Capacity</li> 
<li>Lift Height</li>
<li>Hours</li>
<li>Engine Type</li>
<li>Year</li>
<li>Price</li>
<li>Seller name</li>
<li>Seller location</li>
<li>Equipment location</li>
<li>serial number</li>
<li>link to ad</li>
<li>Auction yes/no</li>
<li>Auction date</li>

### Import Library

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import dataframe_image as dfi

import re

### Create First Request (list by search)

In [77]:
# list pages (search = 'aislemaster')
## title, link, manufacturer, price, location, status, desc, stars, auction

title, links, manufacture, price, location, status, desc, stars, auction = [], [], [], [], [], [], [], [], []

op = webdriver.ChromeOptions()
op.add_argument("--headless=new")
op.add_experimental_option(
    "excludeSwitches", ['enable-automation'])

op.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
# op.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=op)  
driver.get('https://www.machinio.com/');
time.sleep(1) # Let the user actually see something!

login_btn = driver.find_element(By.NAME, "search")

# comment or uncomment below to change search key
# login_btn.send_keys("aislemaster")
login_btn.send_keys("combilift")

login_btn = driver.find_element(By.ID, "search-submit")
login_btn.click()
time.sleep(1) # Let the user actually see something!

# total page number
page_num = driver.find_element(By.ID, 'search-results')
page_num = int(page_num.get_attribute('data-total-pages'))


for i in range(1, page_num+1, 1):
    page_class = "div[data-page-number='{}']".format( i )
    page_class = driver.find_element(By.CSS_SELECTOR, page_class)

    pages = page_class.find_element(By.CLASS_NAME, 'offer-listing__list') 
    daftar = pages.find_elements(By.CLASS_NAME, 'c-card.c-card--link.c-listing-card')
    
    for j in daftar:
        new_tab = None # auction check
        # check post available
        try:
            auctioncek = j.find_element(By.CLASS_NAME, 'lead-wizard-modal-open-button-container').get_attribute('data-contactable') # post availabled
        except:
            auctioncek = 'true'
            new_tab = True
        
        if auctioncek == 'true':
            if new_tab:
                auction.append('Yes')
            else:
                auction.append('No')
            try: 
                title.append(j.find_element(By.CSS_SELECTOR, "h4").text)
            except: title.append('-')
            try: 
                links.append(j.find_element(By.CSS_SELECTOR, "h4 a").get_attribute('href'))
            except: links.append('-')
            try: 
                manufacture.append(j.find_element(By.CLASS_NAME, "c-listing-card__spec-value").text)
            except: manufacture.append('-')
            try: 
                price.append(j.find_element(By.CLASS_NAME, "c-listing-card__price").text)
            except: price.append('-')
            try: 
                location.append(j.find_element(By.CLASS_NAME, "c-card__text-light.c-listing-card__location").text)
            except: location.append('-')
            try: 
                status.append(j.find_element(By.CLASS_NAME, "c-listing-card__spec-used").text)
            except: status.append('-')
            try: 
                desc.append(j.find_element(By.CLASS_NAME, "c-card__text").text)
            except: desc.append('-')
            try:
                stars.append(j.get_attribute('data-star-count'))
            except:
                stars.append('-')
                
    if i == page_num:
        break
    else:
        driver.find_element(By.CLASS_NAME, 'c-pagination__item.icon-chevron-right-solid').click()
        time.sleep(5)

print('FINISH')
driver.quit()

FINISH


### Save DataFrame to Pickle File

In [87]:
df1 = pd.DataFrame({'title': title, 'links': links, 'manufacture': manufacture, 'price': price, 'location': location, 
             'status': status, 'desc': desc, 'ratings': stars, 'auction': auction})

# comment or uncomment this to save different name pickel file per key search
# df1.to_pickle('./aisle_list1.pkl')
df1.to_pickle('./combilift_list1.pkl')

In [38]:
# https://pypi.org/project/dataframe-image/

df = pd.read_pickle('aisle_merged1.pkl')

dfi.export(df, 'df1_aisle.png', max_rows=20)

### Create Function to Scrape Detail Pages

In [68]:
# function detail pages
def get_details(driver1, key_search):
    model = None; cap = None; lh = None; hours = None; ep = None; year = None; price = None; sell_loc = None; eq_loc = None; sn = None;
    # get description
    try:
        desc = driver1.find_element(By.ID, "listing-show-description-body").text.split('\n')
    except: desc = ''
    
    
    # find model
    try:
        model = driver1.find_element(By.XPATH, "//*[contains(text(), 'Model:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('model', aih, re.IGNORECASE):
                    model = aih
            if not model:
                model = driver1.find_element(By.TAG_NAME, 'h1').text
                model = re.split(key_search, model, flags=re.IGNORECASE)[1].split(' ')[1]
        except:model='-'
    
    
    # find capacity
    try:
        cap = driver1.find_element(By.XPATH, "//*[contains(text(), 'Capacity:') or contains(text(), 'capacity:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('capacity', aih, re.IGNORECASE):
                    cap = aih
        except:cap='-'
            
    
    # find lift height
    try:
        lh = driver1.find_element(By.XPATH, "//*[contains(text(), 'Lift height:') or contains(text(), 'Lift Height:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('lift height', aih, re.IGNORECASE):
                    lh = aih
        except:lh='-'
            
    
    # find hours
    try:
        hours = driver1.find_element(By.XPATH, "//*[contains(text(), 'Hours:') or contains(text(), 'Hrs.:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('hours', aih, re.IGNORECASE):
                    hours = aih
        except:hours='-'
    
    
    # find engine type
    try:
        ep = driver1.find_element(By.XPATH, "//*[contains(text(), 'Fuel type:') or contains(text(), 'Power:') or contains(text(), 'Fuel:') or contains(text(), 'Propulsion type:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('fuel type', aih, re.IGNORECASE):
                    ep = aih
                elif re.search('power', aih, re.IGNORECASE):
                    ep = aih
                elif re.search('propulsion type', aih, re.IGNORECASE):
                    ep = aih
                elif re.search('fuel', aih, re.IGNORECASE):
                    ep = aih
        except:ep='-'
    
    
    # find year
    try:
        year = driver1.find_element(By.XPATH, "//*[contains(text(), 'Year:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('year', aih, re.IGNORECASE):
                    year = aih
        except:year='-'
    
    
    # find price
    try:
        price = driver1.find_element(By.XPATH, "//*[contains(text(), 'Price:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('price', aih, re.IGNORECASE):
                    year = aih
        except:price='-'
            
    
    # find seller and equipment location
    try:
        location = driver1.find_element(By.XPATH, "//*[contains(text(), 'Location:')]/following-sibling::dd").text
        sell_loc = location
        eq_loc = location
    except:
        try:
            location = driver1.find_element(By.TAG_NAME, 'h1').text
            location = location.split(' in ')[1]
            sell_loc = location
            eq_loc = location
        except:
            sell_loc = '-'
            eq_loc = '-'
    
    
    # find serial number
    try:
        sn = driver1.find_element(By.XPATH, "//*[contains(text(), 'Serial number:')]/following-sibling::dd").text
    except:
        try:
            for aih in desc:
                if re.search('serial number', aih, re.IGNORECASE):
                    sn = aih
        except:sn='-'

    data = {'model': model,
                    'cap': cap,
                    'lh': lh,
                    'hours': hours,
                    'ep': ep,
                    'year': year,
                    'price': price,
                    'sell_loc': sell_loc,
                    'eq_loc': eq_loc,
                    'sn': sn}
    return data

### Scrape Detail Pages

In [90]:
# get detail pages
data2 = []

op = webdriver.ChromeOptions()
op.add_argument("--headless=new")
op.add_experimental_option(
    "excludeSwitches", ['enable-automation'])

op.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")

driver = webdriver.Chrome(options=op)  
# driver = webdriver.Chrome()  

for i in df1.links:
    driver.get(i);
    time.sleep(1) # Let the user actually see something!

    # comment or uncomment this to save different name pickel file per key search
    # data = get_details(driver, 'aislemaster')
    data = get_details(driver, 'combilift')
    
    time.sleep(1)
    data2.append(data)
print('FINISH')
driver.quit()

FINISH


### Merge DataFrame

In [91]:
# change dict to df
df2 = pd.DataFrame(data2)

# merge df, get used value
df_merged = df1[['title', 'links', 'auction']].copy()
df_merged = pd.concat([df_merged, df2], axis=1).reindex(df_merged.index)

# renaming columns
df_merged.rename(columns={'links': 'link', 'cap': 'capacity', 'lh': 'lift_height', 'ep': 'engine_type', 'sell_loc': 'seller_location',
                         'eq_loc': 'equipment_location', 'sn': 'serial_number'}, inplace=True)

In [92]:
# comment or uncomment this to save different name pickel file per key search
# df2.to_pickle('./aisle_details1.pkl')
# df_merged.to_pickle('./aisle_merged1.pkl')

df2.to_pickle('./combilift_details1.pkl')
df_merged.to_pickle('./combilift1_merged.pkl')

<hr>